In [1]:
import pandas as pd
import numpy as np
import pyreadstat as st
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

In [2]:
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2024 3T\[MT06] Microeconometría I\Clases\Stata\Archivos Stata-20250107\laborsub.dta"

df, meta = st.read_dta(path)
df.head()

,lfp,whrs,kl6,k618,wa,we
0,1,1610,1,0,32,12
1,1,1656,0,2,30,12
2,1,1980,1,3,35,12
3,1,456,0,3,34,12
4,1,1568,1,2,31,14


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   lfp     250 non-null    int64
 1   whrs    250 non-null    int64
 2   kl6     250 non-null    int64
 3   k618    250 non-null    int64
 4   wa      250 non-null    int64
 5   we      250 non-null    int64
dtypes: int64(6)
memory usage: 11.8 KB


In [4]:
meta.column_labels

['1 if woman worked in 1975',
 "Wife's hours of work",
 '# of children younger than 6',
 '# of children between 6 and 18',
 "Wife's age",
 "Wife's educational attainment"]

In [5]:
df['lfp'].value_counts()

lfp
1    150
0    100
Name: count, dtype: int64

In [9]:
df[['whrs']].describe().T

,count,mean,std,min,25%,50%,75%,max
whrs,250.0,799.84,915.60348,0.0,0.0,406.5,1599.75,4950.0


In [10]:
#¿Qué sucede si queremos estimar una función de oferta de trabajo?
import statsmodels.api as sm

# Filtrar datos donde whrs > 0
df_filtered = df[df["whrs"] > 0]

# Definir variables dependiente (Y) e independientes (X)
y = df_filtered["whrs"]
X = df_filtered[["kl6", "k618", "wa", "we"]]

# Agregar constante para la intersección (equivalente a _cons en Stata)
X = sm.add_constant(X)

# Ajustar el modelo
model = sm.OLS(y, X).fit()

# Mostrar resultados
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   whrs   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     2.802
Date:                Thu, 20 Feb 2025   Prob (F-statistic):             0.0281
Time:                        23:26:34   Log-Likelihood:                -1214.6
No. Observations:                 150   AIC:                             2439.
Df Residuals:                     145   BIC:                             2454.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1629.8168    615.130      2.650      0.0

In [12]:
from statsmodels.discrete.count_model import Tobit

# Definir variables dependiente (Y) e independientes (X)
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]

# Agregar constante para la intersección (equivalente a _cons en Stata)
X = sm.add_constant(X)

# Ajustar el modelo Tobit con censura inferior en 0
model = Tobit(y, X).fit(censor_lower=0)

# Mostrar resultados
print(model.summary())

ImportError: cannot import name 'Tobit' from 'statsmodels.discrete.count_model' (c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\discrete\count_model.py)

In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize

# Definir variables dependiente (Y) e independientes (X)
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]

# Agregar constante
X = sm.add_constant(X)

# Función de log-verosimilitud del modelo Tobit (censura inferior en 0)
def tobit_loglik(params, y, X):
    beta = params[:-1]  # Coeficientes de regresión
    sigma = params[-1]  # Desviación estándar del error
    
    if sigma <= 0:
        return np.inf  # Evitar valores no válidos
    
    xb = X @ beta  # Producto matricial X * beta
    uncensored = (y > 0)
    
    # Parte no censurada: Log-verosimilitud de la normal
    ll_uncensored = -0.5 * np.log(2 * np.pi) - np.log(sigma) - (y - xb) ** 2 / (2 * sigma**2)
    
    # Parte censurada: Probabilidad acumulada de la normal (usando scipy.stats.norm)
    ll_censored = np.log(np.maximum(norm.cdf(xb / sigma), 1e-10))  # Evita log(0)
    
    # Combinar log-verosimilitudes
    loglik = np.where(uncensored, ll_uncensored, ll_censored)
    
    return -np.sum(loglik)  # Maximizar la verosimilitud minimizando la función de pérdida

# Estimación de parámetros iniciales
init_params = np.append(np.zeros(X.shape[1]), 1)  # Coeficientes iniciales + sigma

# Optimización numérica
res = minimize(tobit_loglik, init_params, args=(y, X), method="BFGS")

# Resultados
beta_est = res.x[:-1]  # Coeficientes estimados
sigma_est = res.x[-1]  # Estimación de sigma

print("Coeficientes estimados:", beta_est)
print("Sigma estimado:", sigma_est)



c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_numdiff.py:592: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Coeficientes estimados: [-5200401.23618509  1717029.03177521   313985.12206049    94771.03642374
    44524.92114298]
Sigma estimado: 510055.98735519126


In [5]:
import math
import warnings

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.stats
from scipy.special import log_ndtr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


def split_left_right_censored(x, y, cens):
    counts = cens.value_counts()
    if -1 not in counts and 1 not in counts:
        warnings.warn("No censored observations; use regression methods for uncensored data")
    xs = []
    ys = []

    for value in [-1, 0, 1]:
        if value in counts:
            split = cens == value
            y_split = np.squeeze(y[split].values)
            x_split = x[split].values

        else:
            y_split, x_split = None, None
        xs.append(x_split)
        ys.append(y_split)
    return xs, ys


def tobit_neg_log_likelihood(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1])
    s = params[-1]

    to_cat = []

    cens = False
    if y_left is not None:
        cens = True
        left = (y_left - np.dot(x_left, b))
        to_cat.append(left)
    if y_right is not None:
        cens = True
        right = (np.dot(x_right, b) - y_right)
        to_cat.append(right)
    if cens:
        concat_stats = np.concatenate(to_cat, axis=0) / s
        log_cum_norm = scipy.stats.norm.logcdf(concat_stats)  # log_ndtr(concat_stats)
        cens_sum = log_cum_norm.sum()
    else:
        cens_sum = 0

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        mid = scipy.stats.norm.logpdf(mid_stats) - math.log(max(np.finfo('float').resolution, s))
        mid_sum = mid.sum()
    else:
        mid_sum = 0

    loglik = cens_sum + mid_sum

    return - loglik


def tobit_neg_log_likelihood_der(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1]) # in censReg, not using chain rule as below; they optimize in terms of log(s)
    s = params[-1]

    beta_jac = np.zeros(len(b))
    sigma_jac = 0

    if y_left is not None:
        left_stats = (y_left - np.dot(x_left, b)) / s
        l_pdf = scipy.stats.norm.logpdf(left_stats)
        l_cdf = log_ndtr(left_stats)
        left_frac = np.exp(l_pdf - l_cdf)
        beta_left = np.dot(left_frac, x_left / s)
        beta_jac -= beta_left

        left_sigma = np.dot(left_frac, left_stats)
        sigma_jac -= left_sigma

    if y_right is not None:
        right_stats = (np.dot(x_right, b) - y_right) / s
        r_pdf = scipy.stats.norm.logpdf(right_stats)
        r_cdf = log_ndtr(right_stats)
        right_frac = np.exp(r_pdf - r_cdf)
        beta_right = np.dot(right_frac, x_right / s)
        beta_jac += beta_right

        right_sigma = np.dot(right_frac, right_stats)
        sigma_jac -= right_sigma

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        beta_mid = np.dot(mid_stats, x_mid / s)
        beta_jac += beta_mid

        mid_sigma = (np.square(mid_stats) - 1).sum()
        sigma_jac += mid_sigma

    combo_jac = np.append(beta_jac, sigma_jac / s)  # by chain rule, since the expression above is dloglik/dlogsigma

    return -combo_jac


class TobitModel:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.ols_coef_ = None
        self.ols_intercept = None
        self.coef_ = None
        self.intercept_ = None
        self.sigma_ = None

    def fit(self, x, y, cens, verbose=False):
        """
        Fit a maximum-likelihood Tobit regression
        :param x: Pandas DataFrame (n_samples, n_features): Data
        :param y: Pandas Series (n_samples,): Target
        :param cens: Pandas Series (n_samples,): -1 indicates left-censored samples, 0 for uncensored, 1 for right-censored
        :param verbose: boolean, show info from minimization
        :return:
        """
        x_copy = x.copy()
        if self.fit_intercept:
            x_copy.insert(0, 'intercept', 1.0)
        else:
            x_copy.scale(with_mean=True, with_std=False, copy=False)
        init_reg = LinearRegression(fit_intercept=False).fit(x_copy, y)
        b0 = init_reg.coef_
        y_pred = init_reg.predict(x_copy)
        resid = y - y_pred
        resid_var = np.var(resid)
        s0 = np.sqrt(resid_var)
        params0 = np.append(b0, s0)
        xs, ys = split_left_right_censored(x_copy, y, cens)

        result = minimize(lambda params: tobit_neg_log_likelihood(xs, ys, params), params0, method='BFGS',
                          jac=lambda params: tobit_neg_log_likelihood_der(xs, ys, params), options={'disp': verbose})
        if verbose:
            print(result)
        self.ols_coef_ = b0[1:]
        self.ols_intercept = b0[0]
        if self.fit_intercept:
            self.intercept_ = result.x[1]
            self.coef_ = result.x[1:-1]
        else:
            self.coef_ = result.x[:-1]
            self.intercept_ = 0
        self.sigma_ = result.x[-1]
        return self

    def predict(self, x):
        return self.intercept_ + np.dot(x, self.coef_)

    def score(self, x, y, scoring_function=mean_absolute_error):
        y_pred = np.dot(x, self.coef_)
        return scoring_function(y, y_pred)

In [ ]:
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(X)
df['cens'] = np.where(df['whrs'] == 0, -1, 0)
cens = df['cens']
tr = TobitModel()
tr = tr.fit(X, y, cens, verbose=False)
tr.coef_

array([ 294.44827939, -827.78090417, -140.01430074,  -24.97854148,
        103.69518044])

In [8]:
df["whrs"] = df["whrs"].apply(lambda x: max(x, 0))
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(X)
df['cens'] = np.where(df['whrs'] == 0, 0, 0)
cens = df['cens']
tr = TobitModel()
tr = tr.fit(X, y, cens, verbose=False)
tr.coef_

C:\Users\HP\AppData\Local\Temp\ipykernel_7068\1321454343.py:16: UserWarning: No censored observations; use regression methods for uncensored data
  warnings.warn("No censored observations; use regression methods for uncensored data")


array([ 470.02964199, -462.12329817,  -91.14099631,  -13.15770391,
         53.26155967])

In [23]:
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

class Tobit(GenericLikelihoodModel):
    def __init__(self, endog, exog, left=0, **kwds):
        self.left = left
        super().__init__(endog, exog, **kwds)

    def loglike(self, params):
        # Separar beta (coeficientes) y sigma (desviación estándar)
        beta = params[:-1]  # Los coeficientes
        sigma = params[-1]  # La desviación estándar (último parámetro)
        
        # Predicciones lineales
        xb = np.dot(self.exog, beta)
        y = self.endog

        # Calcular la log-verosimilitud
        ll = np.where(y > self.left,
                      np.log(1 / sigma) + sm.distributions.norm.logpdf((y - xb) / sigma),
                      sm.distributions.norm.logcdf((self.left - xb) / sigma))
        
        return ll.sum()

# Cargar datos
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
y[y < 0] = 0  # Censuramos en 0

# Ajustar modelo
X = sm.add_constant(X)
model = Tobit(y, X, left=0)
result = model.fit()
print(result.summary())

C:\Users\HP\AppData\Local\Temp\ipykernel_21004\3479087987.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y < 0] = 0  # Censuramos en 0


ValueError: shapes (250,5) and (4,) not aligned: 5 (dim 1) != 4 (dim 0)

In [ ]:
from py4etrics.tobit import Tobit
import statsmodels.api as sm

y = df["whrs"]
x = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(x)
df['cens'] = np.where(df['whrs'] == 0, -1, 0)
cens = df['cens']
model = Tobit(y, X, cens).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 5.468361
         Iterations: 566
         Function evaluations: 894


<class 'statsmodels.iolib.summary.Summary'>
"""
                              Tobit Regression Results                             
===================================================================================
Dep. Variable:                        whrs   Pseudo R-squ:                    0.008
Method:                 Maximum Likelihood   Log-Likelihood:                -1367.1
No. Observations:                      250   LL-Null:                       -1378.6
No. Uncensored Obs:                    150   LL-Ratio:                         23.0
No. Left-censored Obs:                 100   LLR p-value:                     0.000
No. Right-censored Obs:                  0   AIC:                            2744.2
Df Residuals:                          245   BIC:                            2761.8
Df Model:                                4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        589.0003    841.589      0.700      0.484   -1060.483    2238.484
kl6         -827.7658    214.752     -3.855      0.000   -1248.672    -406.860
k618        -140.0192     74.227     -1.886      0.059    -285.502       5.463
wa           -24.9792     13.257     -1.884      0.060     -50.963       1.004
we           103.6896     41.826      2.479      0.013      21.712     185.668
Log(Sigma)     7.1777      0.063    113.628      0.000       7.054       7.302
==============================================================================
"""

In [3]:
from py4etrics.truncreg import Truncreg
import statsmodels.api as sm

y = df["whrs"]
x = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(x)
model = Truncreg(y, X, left=0).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 7.658411
         Iterations: 4994
         Function evaluations: 7886


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                         Truncreg Regression Results                          
==============================================================================
Dep. Variable:                   whrs   Pseudo R-squ:                    0.003
Model:                       Truncreg   Log-Likelihood:                -1914.6
Method:            Maximum Likelihood   LL-Null:                       -1921.1
Date:                Sat, 22 Feb 2025   LL-Ratio:                         13.0
Time:                        23:35:32   LLR p-value:                     0.011
No. Observations:                 250   AIC:                            3839.2
Df Residuals:                     245   BIC:                            3856.8
Df Model:                           4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.664e+10        nan        nan        nan         nan         nan
kl6        -5.197e+09        nan        nan        nan         nan         nan
k618       -1.261e+08        nan        nan        nan         nan         nan
wa         -8.649e+07        nan        nan        nan         nan         nan
we          1.056e+08        nan        nan        nan         nan         nan
Log(Sigma)    15.1967        nan        nan        nan         nan         nan
==============================================================================
"""

In [5]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import Bounds, minimize
import re

def truncreg(formula, data, point, direction, scaled=False, iterlim=50):
    if direction not in ["left", "right"]:
        raise ValueError("'direction' must be 'left' or 'right'")
    
    x_vars = re.split(r'[*+]', re.search(r'~\s*(.+)', formula).group(1).strip())
    x_vars = [var.strip() for var in x_vars]
    x = data[[var for var in x_vars if var]]
    y = data[formula.split('~')[0].strip()]
    
    intercept = np.ones((x.shape[0], 1))
    x = np.hstack((intercept, x))
    
    if direction == "left" and any(y < point):
        raise ValueError("Data non-truncated, contains observations < '{}'".format(point))
    if direction == "right" and any(y > point):
        raise ValueError("Data non-truncated, contains observations > '{}'".format(point))

    def maxLikTruncreg(param, x, y, point, direction, scaled=scaled):
        epsilon = 1e-8
        sign = 1 if direction == "left" else -1
        beta = param[:-1]
        sigma = param[-1]
        bX = np.dot(x, beta)
        
        if not scaled:
            resid = y - bX
            trunc = bX - point
    
            #Potential Overflow Handling
            exp_argument = norm.logpdf(sign * trunc / (sigma + epsilon)) - norm.logcdf(sign * trunc / (sigma + epsilon))
            exp_argument = np.clip(exp_argument, a_min=None, a_max=700)
            mills = np.exp(exp_argument)
    
            lnL = -np.log(sigma + epsilon) + norm.logpdf(resid / (sigma + epsilon)) - norm.logcdf(sign * trunc / (sigma + epsilon))
            logLik = lnL.sum() if isinstance(lnL, np.ndarray) and lnL.ndim > 0 else lnL
    
            # Gradient calculation
            gbX = resid / (sigma ** 2 + epsilon) - sign * mills / (sigma + epsilon)
            gsigma = -1 / (sigma + epsilon) + resid ** 2 / (sigma + epsilon) ** 3 + sign * mills * trunc / (sigma + epsilon) ** 2
            gradient = np.concatenate([gbX, gsigma])
    
            # Hessian calculation
            bb = -1 / (sigma ** 2 + epsilon) + mills * (sign * trunc / (sigma + epsilon) + mills) / (sigma + epsilon) ** 2
            ss = 1 / (sigma + epsilon) ** 2 - 3 * resid ** 2 / (sigma + epsilon) ** 4 - 2 * mills * sign * trunc / (sigma + epsilon) ** 3 + \
                 mills * (sign * trunc / (sigma + epsilon) + mills) * trunc ** 2 / (sigma + epsilon) ** 4
            bs = -2 * resid / (sigma + epsilon) ** 3 + sign * mills / (sigma + epsilon) ** 2 - \
                 mills * (mills + sign * trunc / (sigma + epsilon)) * trunc / (sigma + epsilon) ** 3
            bs = np.array(bs)
    
            bb_matrix = np.dot(x.T, np.reshape(bb, (-1, 1)) * x)
            bs_vector = np.sum(np.reshape(bs, (-1, 1)) * x, axis=0)
            bs_vector = np.reshape(bs_vector, (-1, 1))
            ss_scalar = np.array([[np.sum(ss)]])
    
            hessian = np.block([[bb_matrix, bs_vector],
                            [bs_vector.T, ss_scalar]])
        else:
            exp_argument = norm.logpdf(sign * (bX - point / (sigma + epsilon))) - norm.logcdf(sign * (bX - point / (sigma + epsilon)))
            exp_argument = np.clip(exp_argument, a_min=None, a_max=700)
            mills = np.exp(exp_argument)
            
            lnL = -np.log(sigma + epsilon) + norm.logpdf(y / (sigma + epsilon) - bX) - norm.logcdf(sign * (bX - point / (sigma + epsilon)))
            logLik = lnL.sum() if isinstance(lnL, np.ndarray) and lnL.ndim > 0 else lnL
            
            #Gradient calculation
            gbX = (y / sigma - bX + epsilon) - mills * sign
            gsigma = -1 / (sigma + epsilon) + (y / sigma + epsilon - bX) * y /(sigma + epsilon) ** 2 - sign * mills * point/(sigma + epsilon) ** 2
            gradient = np.concatenate([gbX, gsigma])
            
            #Hessian Calculation
            bb = -1 + mills * (mills + sign * (bX - point / (sigma + epsilon)))
            bs = -y / sigma ** 2 + (mills + sign * (bX - point / (sigma + epsilon))) * mills * point / (sigma + epsilon) ** 2
            ss = 1 / (sigma + epsilon) ** 2 - 3 * y ** 2 / (sigma + epsilon) ** 4 + 2 * bX * y / (sigma + epsilon) ** 3 + \
                mills * (mills + sign * (bX - point / (sigma + epsilon))) * point ** 2 / (sigma + epsilon) ** 4 + \
                2 * sign * mills * point / (sigma + epsilon) ** 3
            bs = np.array(bs)
            
            bb_matrix = np.dot(x.T, np.reshape(bb, (-1, 1)) * x)
            bs_vector = np.sum(np.reshape(bs, (-1, 1)) * x, axis=0)
            bs_vector = np.reshape(bs_vector, (-1, 1))
            ss_scalar = np.array([[np.sum(ss)]])
            
            hessian = np.block([[bb_matrix, bs_vector],
                            [bs_vector.T, ss_scalar]])
            
        return {
            'logLik': logLik, 
            'gradient': gradient, 
            'hessian': hessian
        }

    def objective(param):
        result = maxLikTruncreg(param, x, y, point, direction)
        return -result['logLik']

    linmod = np.linalg.lstsq(x, y, rcond=None)
    start_beta = linmod[0]
    start_sigma = np.array([np.std(linmod[1])])
    start = np.concatenate([start_beta, start_sigma])

    lower_bounds = [np.NINF] * (len(start) - 1) + [1e-4]
    upper_bounds = [np.inf] * len(start)
    bounds = Bounds(lower_bounds, upper_bounds)

    result = minimize(objective, start, method='L-BFGS-B', bounds=bounds, options={'maxiter': iterlim})
    opt_result = maxLikTruncreg(result.x, x, y, point, direction, scaled)
    vcov = -np.linalg.inv(opt_result['hessian'])
    std_errors = np.sqrt(np.diag(vcov))
    
    return {
        'result': result,
        'vcov': vcov,
        'SE': std_errors
    }

In [13]:
result = truncreg(formula='whrs~kl6+k618+wa+we', data=df, point=0, direction='left')
print(result)

{'result':   message: STOP: TOTAL NO. of ITERATIONS REACHED LIMIT
  success: False
   status: 1
      fun: 2108.38559242432
        x: [ 9.393e+02 -4.677e+02 -9.946e+01 -1.575e+01  5.119e+01
             5.322e+02]
      nit: 50
      jac: [-1.723e-02  3.402e-02  2.765e-02 -1.001e+00 -2.767e-01
            -9.420e-01]
     nfev: 455
     njev: 65
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>, 'vcov': array([[ 1.43858120e+05, -1.25420225e+04, -6.10188647e+03,
        -1.73430874e+03, -4.60015031e+03, -1.07771422e+02],
       [-1.25420225e+04,  1.14460624e+04,  4.09661983e+02,
         2.69847936e+02, -1.29942821e+02,  1.01826624e+02],
       [-6.10188647e+03,  4.09661983e+02,  1.26477413e+03,
         9.35012849e+01,  3.27066443e+01,  5.75536589e+00],
       [-1.73430874e+03,  2.69847936e+02,  9.35012849e+01,
         3.54375023e+01,  3.49471335e+00, -6.66373172e-01],
       [-4.60015031e+03, -1.29942821e+02,  3.27066443e+01,
         3.49471335e+00,  3.54336606e+02, -3.298341

In [14]:
result

{'result':   message: STOP: TOTAL NO. of ITERATIONS REACHED LIMIT
   success: False
    status: 1
       fun: 2108.38559242432
         x: [ 9.393e+02 -4.677e+02 -9.946e+01 -1.575e+01  5.119e+01
              5.322e+02]
       nit: 50
       jac: [-1.723e-02  3.402e-02  2.765e-02 -1.001e+00 -2.767e-01
             -9.420e-01]
      nfev: 455
      njev: 65
  hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>,
 'vcov': array([[ 1.43858120e+05, -1.25420225e+04, -6.10188647e+03,
         -1.73430874e+03, -4.60015031e+03, -1.07771422e+02],
        [-1.25420225e+04,  1.14460624e+04,  4.09661983e+02,
          2.69847936e+02, -1.29942821e+02,  1.01826624e+02],
        [-6.10188647e+03,  4.09661983e+02,  1.26477413e+03,
          9.35012849e+01,  3.27066443e+01,  5.75536589e+00],
        [-1.73430874e+03,  2.69847936e+02,  9.35012849e+01,
          3.54375023e+01,  3.49471335e+00, -6.66373172e-01],
        [-4.60015031e+03, -1.29942821e+02,  3.27066443e+01,
          3.49471335e+00,  3.54